In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTNeoXForCausalLM, AutoConfig
from accelerate import init_empty_weights
import json 

# with open('device_map.json', 'r') as f:
#     device_map = json.load(f)
    
checkpoint = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
cache_dir='cache'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir=cache_dir)
# model = GPTNeoXForCausalLM.from_pretrained(checkpoint, cache_dir=cache_dir).half().cuda()
# model = GPTNeoXForCausalLM.from_pretrained(checkpoint, cache_dir=cache_dir, device_map=device_map)#.half()
model = GPTNeoXForCausalLM.from_pretrained(checkpoint, cache_dir=cache_dir, device_map="auto").half()
# model.tie_weights()



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
model.hf_device_map

{'gpt_neox.embed_in': 0,
 'gpt_neox.layers.0': 0,
 'gpt_neox.layers.1': 0,
 'gpt_neox.layers.2': 0,
 'gpt_neox.layers.3': 0,
 'gpt_neox.layers.4': 0,
 'gpt_neox.layers.5': 0,
 'gpt_neox.layers.6': 0,
 'gpt_neox.layers.7': 0,
 'gpt_neox.layers.8': 1,
 'gpt_neox.layers.9': 1,
 'gpt_neox.layers.10': 1,
 'gpt_neox.layers.11': 1,
 'gpt_neox.layers.12': 1,
 'gpt_neox.layers.13': 1,
 'gpt_neox.layers.14': 1,
 'gpt_neox.layers.15': 1,
 'gpt_neox.layers.16': 1,
 'gpt_neox.layers.17': 1,
 'gpt_neox.layers.18': 2,
 'gpt_neox.layers.19': 2,
 'gpt_neox.layers.20': 2,
 'gpt_neox.layers.21': 2,
 'gpt_neox.layers.22': 2,
 'gpt_neox.layers.23': 2,
 'gpt_neox.layers.24': 2,
 'gpt_neox.layers.25': 2,
 'gpt_neox.layers.26': 2,
 'gpt_neox.layers.27': 2,
 'gpt_neox.layers.28': 3,
 'gpt_neox.layers.29': 3,
 'gpt_neox.layers.30': 3,
 'gpt_neox.layers.31': 3,
 'gpt_neox.layers.32': 3,
 'gpt_neox.layers.33': 3,
 'gpt_neox.layers.34': 3,
 'gpt_neox.layers.35': 3,
 'gpt_neox.final_layer_norm': 3,
 'embed_out': 3}

In [3]:
# save json
# import json 
# with open('device_map.json', 'w') as f:
#     json.dump(model.hf_device_map, f)

In [4]:
inputs = tokenizer("<|prompter|>What is a meme, and what's the history behind this word?<|endoftext|><|assistant|>", return_tensors="pt")
inputs = inputs.to(0)
tokens = model.generate(**inputs, do_sample=True, temperature=0.9, max_length=1024)
tokenizer.decode(tokens[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'What is a meme, and what\'s the history behind this word?A meme, or cultural joke that spreads virally, is a unit of cultural information that is mimicked and passed on from person to person. It is often humorous and can involve various forms of media such as images and GIFs. The word "meme" comes from the Greek word "mimema", which means "to imitate". The first known use of "meme" was in 1955, in a book by Richard Dawkins called "The Selfish Gene".'